In [1]:
print("Name: SUKRUTHA D")
print("SRN: PES2UG23CS622")

Name: SUKRUTHA D
SRN: PES2UG23CS622


**Part 3a: Chain of Thought (CoT)**

In [2]:
%pip install python-dotenv --upgrade --quiet langchain langchain-groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.7/111.7 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 500.5/500.5 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.1/158.1 kB 11.2 MB/s eta 0:00:00


In [4]:
from dotenv import load_dotenv
load_dotenv()

import getpass
import os
from langchain_groq import ChatGroq
if "GROQ_API_KEY" not in os.environ:
    os.environ["GROQ_API_KEY"] = getpass.getpass("Enter your Groq API Key: ")

llm = ChatGroq(model="llama-3.1-8b-instant", temperature=0.0)


In [5]:
question = "Roger has 5 tennis balls. He buys 2 more cans of tennis balls. Each can has 3 tennis balls. How many does he have now?"


prompt_standard = f"Answer this question: {question}"
print("--- STANDARD (Llama3.1-8b) ---")
print(llm.invoke(prompt_standard).content)

--- STANDARD (Llama3.1-8b) ---
To find out how many tennis balls Roger has now, we need to add the initial number of tennis balls he had (5) to the number of tennis balls he bought (2 cans * 3 tennis balls per can).

2 cans * 3 tennis balls per can = 6 tennis balls

Now, let's add the initial number of tennis balls (5) to the number of tennis balls he bought (6):

5 + 6 = 11

So, Roger now has 11 tennis balls.


In [6]:
prompt_cot = f"Answer this question. Let's think step by step. {question}"

print("--- Chain of Thought (Llama3.1-8b) ---")
print(llm.invoke(prompt_cot).content)

--- Chain of Thought (Llama3.1-8b) ---
To find out how many tennis balls Roger has now, we need to follow these steps:

1. Roger already has 5 tennis balls.
2. He buys 2 more cans of tennis balls. Each can has 3 tennis balls, so he buys 2 x 3 = 6 more tennis balls.
3. Now, we add the tennis balls he already had (5) to the new tennis balls he bought (6). 5 + 6 = 11

So, Roger now has 11 tennis balls.


**Part 3b: Tree of Thoughts (ToT) & Graph of Thoughts(GoT)**

In [7]:
if "GROQ_API_KEY" not in os.environ:
    os.environ["GROQ_API_KEY"] = getpass.getpass("Enter your Groq API Key: ")

llm = ChatGroq(model="llama-3.1-8b-instant", temperature=0.7)


In [8]:
#Tree Of Thoughts
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnableLambda
from langchain_core.output_parsers import StrOutputParser

problem = "How can I get my 5-year-old to eat vegetables?"

prompt_branch = ChatPromptTemplate.from_template(
    "Problem: {problem}. Give me one unique, creative solution. Solution {id}:"
)

branches = RunnableParallel(
    sol1=prompt_branch.partial(id="1") | llm | StrOutputParser(),
    sol2=prompt_branch.partial(id="2") | llm | StrOutputParser(),
    sol3=prompt_branch.partial(id="3") | llm | StrOutputParser(),
)

prompt_judge = ChatPromptTemplate.from_template(
    """
    I have three proposed solutions for: '{problem}'

    1: {sol1}
    2: {sol2}
    3: {sol3}

    Act as a Child Psychologist. Pick the most sustainable one (not bribery) and explain why.
    """
)
tot_chain = (
    RunnableParallel(problem=RunnableLambda(lambda x: x), branches=branches)
    | (lambda x: {**x["branches"], "problem": x["problem"]})
    | prompt_judge
    | llm
    | StrOutputParser()
)
print("--- Tree of Thoughts (ToT) Result ---")
print(tot_chain.invoke(problem))

--- Tree of Thoughts (ToT) Result ---
After reviewing the three proposed solutions, I would recommend Solution 2: Make it a Game or a Challenge as the most sustainable option. Here's why:

1. **Encourages exploration, not just consumption**: By turning mealtime into a game or challenge, you're encouraging your child to explore new vegetables in a fun and interactive way. This approach focuses on the process of trying new foods, rather than just consuming them.

2. **Develops problem-solving skills**: Games and challenges require children to think critically and come up with solutions. In this case, they'll need to figure out which vegetables to try, and how to approach them.

3. **Builds confidence and self-efficacy**: As your child successfully tries new vegetables and completes challenges, they'll develop a sense of confidence and self-efficacy. This will help them feel more empowered to make healthy choices.

4. **Long-term benefits**: Unlike the "Veggie Face" approach, which might 

In [9]:
#Graph of Thoughts(GoT)
prompt_draft = ChatPromptTemplate.from_template(
    "Write a 1-sentence movie plot about: {topic}. Genre: {genre}."
)

drafts = RunnableParallel(
    draft_scifi=prompt_draft.partial(genre="Sci-Fi") | llm | StrOutputParser(),
    draft_romance=prompt_draft.partial(genre="Romance") | llm | StrOutputParser(),
    draft_horror=prompt_draft.partial(genre="Horror") | llm | StrOutputParser(),
)

prompt_combine = ChatPromptTemplate.from_template(
    """
    I have three movie ideas for the topic '{topic}':
    1. Sci-Fi: {draft_scifi}
    2. Romance: {draft_romance}
    3. Horror: {draft_horror}

    Your task: Create a new Mega-Movie that combines the TECHNOLOGY of Sci-Fi, the PASSION of Romance, and the FEAR of Horror.
    Write one paragraph.
    """
)

got_chain = (
    RunnableParallel(topic=RunnableLambda(lambda x: x), drafts=drafts)
    | (lambda x: {**x["drafts"], "topic": x["topic"]})
    | prompt_combine
    | llm
    | StrOutputParser()
)
print("--- Graph of Thoughts (GoT) Result ---")
print(got_chain.invoke("Time Travel"))

--- Graph of Thoughts (GoT) Result ---
In the mind-bending thriller, "Chrono Labyrinth," a brilliant scientist, Emma, discovers a way to manipulate the space-time continuum using a revolutionary technology that allows her to travel through the fabric of time. As she experiments with this power, she finds herself falling deeply in love with a younger version of her current boyfriend, Jack, whom she encounters during her time travels. However, their idyllic romance is disrupted when Emma realizes that each time she tries to alter their past, the consequences become increasingly terrifying: every time she changes something, a dark and twisted version of Jack emerges, forcing Emma to fight for control of her own reality and confront the evil doppelganger that threatens to destroy her life. As the stakes escalate and the labyrinthine timelines begin to collapse, Emma must navigate the treacherous landscape of time to find a way to restore the original timeline and save the love of her life.